In [2]:
from m2_functions import *
import numpy as np
import re

In [3]:
def parse_data(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    # Split the content into individual data points
    data_points = content.split('\n\n')

    # Initialize lists to store the parsed data
    variables_list = []
    measured_variables_list = []
    constants_list = []
    non_measured_variables_list = []
    equations_list = []
    derivatives_list = []

    # Regular expressions to extract the relevant information
    variables_pattern = re.compile(r'Variables:\s*(.*?)\n', re.DOTALL)
    measured_variables_pattern = re.compile(r'Measured Variables:\s*(.*?)\n', re.DOTALL)
    constants_pattern = re.compile(r'Constants:\s*(.*?)\n', re.DOTALL)
    non_measured_variables_pattern = re.compile(r'Non Measured Variables:\s*(.*?)\n', re.DOTALL)
    equations_pattern = re.compile(r'Equations:\s*(.*?)(?=\n\s*\d+:|\Z)', re.DOTALL)
    derivatives_pattern = re.compile(r'Derivatives:\s*(.*?)\n', re.DOTALL)

    for data_point in data_points:
        # Extract variables
        variables_match = variables_pattern.search(data_point)
        if variables_match:
            variables = variables_match.group(1).strip().split(',')
            variables_list.append(variables)

        # Extract measured variables
        measured_variables_match = measured_variables_pattern.search(data_point)
        if measured_variables_match:
            measured_variables = measured_variables_match.group(1).strip().split(',')
            measured_variables_list.append(measured_variables)

        # Extract constants
        constants_match = constants_pattern.search(data_point)
        if constants_match:
            constants = constants_match.group(1).strip().split(',')
            constants_list.append(constants)

        # Extract non-measured variables
        non_measured_variables_match = non_measured_variables_pattern.search(data_point)
        if non_measured_variables_match:
            non_measured_variables = non_measured_variables_match.group(1).strip().split(',')
            non_measured_variables_list.append(non_measured_variables)

        # Extract equations
        equations_match = equations_pattern.search(data_point)
        if equations_match:
            equations = equations_match.group(1).strip().split('\n')
            equations = [eq.strip() for eq in equations if eq.strip()]  # Remove empty lines
            equations_list.append(equations)

        # Extract derivatives
        derivatives_match = derivatives_pattern.search(data_point)
        if derivatives_match:
            derivatives = derivatives_match.group(1).strip().split(',')
            derivatives_list.append(derivatives)

    return {
        'variables': variables_list,
        'measured_variables': measured_variables_list,
        'constants': constants_list,
        'non_measured_variables': non_measured_variables_list,
        'equations': equations_list,
        'derivatives': derivatives_list
    }


In [4]:
def extract_variables(equation):
    # Use regex to find all variable-like patterns
    # This matches sequences of letters and numbers, but excludes standalone numbers
    variables = re.findall(r'\b[a-zA-Z_][a-zA-Z0-9_]*\b', equation)
    
    # Remove duplicates by converting to a set, then back to a list
    return list(set(variables))

def get_variables_from_equations(equations):
    # Apply extract_variables to each equation in the list
    return [extract_variables(eq) for eq in equations]

In [5]:
# Path to the text file
file_path = 'synthetic_axioms.txt'

# Parse the data
parsed_data = parse_data(file_path)

# Example: Accessing the first data point
print("Variables:", parsed_data['variables'][56])
print("Measured Variables:", parsed_data['measured_variables'][56])
print("Constants:", parsed_data['constants'][56])
print("Non-Measured Variables:", parsed_data['non_measured_variables'][56])
print("Equations:", parsed_data['equations'][56])
print("Derivatives:", parsed_data['derivatives'][56])

Variables: ['Fc', 'Fg', 'W', 'd1', 'd2', 'm1', 'm2', 'p', 'T']
Measured Variables: ['Fc', 'Fg', 'W', 'd1', 'd2', 'm1', 'm2', 'p', 'T']
Constants: ['G', 'c']
Non-Measured Variables: ['Equations:']
Equations: ['-c*d1*m2*p + 3*Fg*d1^2*m1 - W*d1*m1 + W*d2*m1', '-G*Fg + 2*c^2*d2xdt2*d1', '-2*c*p + d2xdt2*d1*m2 + Fc*d1 - Fc*d2', 'Fc – Fg']
Derivatives: ['d2xdt2']


In [8]:
# Iterate through all the data points and run the projection function

n = len(parsed_data['variables'])

for i in range(n):
    variables = parsed_data['variables'][i] + parsed_data['constants'][i] + parsed_data['derivatives'][i]
    equations = parsed_data['equations'][i]
    axiom_variables = get_variables_from_equations(equations)

    np.random.shuffle(variables)
    j=0
    while j < len(variables):
        temp_measured_variables = variables[:j+1]
        non_measured_variables = list(set(variables) - set(temp_measured_variables))


        if any(set(temp_measured_variables).issubset(set(axiom_vars)) for axiom_vars in axiom_variables) or any(set(axiom_vars).issubset(set(temp_measured_variables)) for axiom_vars in axiom_variables):
            if j == len(variables) - 1:
                j = 0
                np.random.shuffle(variables)
            j+=1
            continue

        projection(variables, equations, temp_measured_variables, non_measured_variables, filename='temp.txt')

        with open('temp.txt', 'r') as temp_file:
            content = temp_file.read()
            os.remove('temp.txt')
            if "matrix {}" not in content:

                if not os.path.exists('target_polynomials'):
                    os.makedirs('target_polynomials')
                with open('target_polynomials/system{}.txt'.format(i), 'w') as file:

                    file.write("System number {}\n".format(i))
                    file.write(f"Measured Variables: {temp_measured_variables}\n")
                    file.write(f"Non-Measured Variables: {non_measured_variables}\n")
                    file.write("Equations:\n")
                    for eq in equations:
                        file.write(eq)
                        file.write("\n")
                    file.write("\n")
                    file.write("Target Polynomial:\n")
                    write = False
                    for line in content.split("\n"):
                        if write:
                            file.write(line)
                        if "Polynomials of the Grobner basis of the eliminated ideal:" in line:
                            write = True
                break
            else:
                if j == len(variables) - 1:
                    j = 0
                    np.random.shuffle(variables)
            
        j += 1


Output from Macaulay2:
Ring defined: R
Axioms defined: {2*d2xdt2*m2-Fc, m1*c-c*m2+p, Fc*d2-W-Fg*d1+2*Fg*d2, d2xdt2^2*d1*d2-d2xdt2*G*m1+Fc*G}
Measured variables defined: {d2xdt2, Fc, W}
Non-measured variables defined: {p, m1, T, c, d2, G, Fg, m2, d1}
Ideal defined: ideal(2*d2xdt2*m2-Fc,m1*c-c*m2+p,Fc*d2-W-Fg*d1+2*Fg*d2,d2xdt2^2*d1*d2-d2xdt2*G*m1+Fc*G)
Gröbner basis: matrix {{m1*c-c*m2+p, W^2*d1-2*W*G*m1*m2+4*W*G*m2^2+2*W*Fg*d1^2-4*W*Fg*d1*d2-2*G*m1*Fg*m2*d1+4*G*m1*Fg*m2*d2+4*G*Fg*m2^2*d1-8*G*Fg*m2^2*d2+Fg^2*d1^3-4*Fg^2*d1^2*d2+4*Fg^2*d1*d2^2, Fc*d2-W-Fg*d1+2*Fg*d2, Fc*W*d1-2*Fc*G*m1*m2+4*Fc*G*m2^2+Fc*Fg*d1^2-2*W*Fg*d1-2*Fg^2*d1^2+4*Fg^2*d1*d2, 2*d2xdt2*m2-Fc, d2xdt2*W*d1+d2xdt2*Fg*d1^2-2*d2xdt2*Fg*d1*d2-Fc*G*m1+2*Fc*G*m2, d2xdt2^2*d1*d2-d2xdt2*G*m1+Fc*G}}
Ring of Measured Variables defined: Rproj
Eliminated Ideal: ideal()
Grobner basis of the eliminated ideal: matrix {}

Output from Macaulay2:
Ring defined: R
Axioms defined: {2*d2xdt2*m2-Fc, m1*c-c*m2+p, Fc*d2-W-Fg*d1+2*Fg*d2, d2xdt2^2*